<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2016</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.0</h1>
<h1>Issued Monday 10/03/2016; Due Monday 10/17/2016</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test your approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for each class meeting after their due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Fill in the fields as described
class Student:
    first = "Calvin"
    last = "Barker"
    onyen = "barkerc1"
    pid = "720391993"

In this problem set we will revisit the *Monkepo* data discussed in Lecture 3. You may want to revist that lecture to reaquaint yourself with the data set. Your first task will be to construct and populate a database with the following schema.

 <table style="border:none;"><tbody>
        <tr style="border:none;">
            <td style="padding:0px 20px;border:none;" colspan="2">
            <table><tbody>
            <tr><th colspan="6" style="text-align:center;">Monkepo</th></tr>
            <tr><td><u>mid</u></td>
                <td>name</td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Appearance</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>latitude</u></td>
                <td><u>longitude</u></td>
                <td><u>datetime</u></td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Class</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>major (1/0)</u></td>
                <td>type</td>
            </tr>
            </tbody></table>
            </td>
        </tr>
        </tbody></table>

The *Monkepo* relation should hold just the *unique* name of a *monkepo* species and an autoincremented primary key, *mid*. When a *Monkepo* appears its location and appearance time are stored as a tuple in the *Appearance* relation. Finally, the *Class* relation is used to hold information about the major and minor class of the monkepo species. The major class is indicated by a value of *1* in the major field, and the minor class by a *0*. There should be no class entry for a *None* minor type.  

Use this verison of the <a href="http://csbio.unc.edu/mcmillan/Media/monkepo.csv" download>data file</a> to fill your database (it is different than the one used in lecture 3).

---
**Part 1:** In the cell provided below write a Python fragment to create the specified database, and name it *"monkepo.db"* (Hint: you should investigate Python's *DateTime* module to find methods for combining the date and time values from the data file into a single attribute). You code should commit and close your database.

In [17]:
import sqlite3, datetime
import pandas as pd

monkepo = sqlite3.connect("monkepo.db")
cur = monkepo.cursor()
cur.execute("""DROP TABLE IF EXISTS Monkepo""")
cur.execute("""DROP TABLE IF EXISTS Appearance""")
cur.execute("""DROP TABLE IF EXISTS Class""")
cur.execute("""CREATE TABLE Monkepo (
                mid INTEGER PRIMARY KEY, 
                name TEXT);""")
cur.execute("""CREATE TABLE Appearance (
                mid INTEGER,
                latitude REAL,
                longitude REAL,
                datetime DATETIME,
                PRIMARY KEY(mid, latitude, longitude, datetime),
                FOREIGN KEY(mid) REFERENCES Monkepo(mid));""")
cur.execute("""CREATE TABLE Class (
                mid INTEGER,
                major INTEGER,
                type TEXT,
                PRIMARY KEY(mid, major),
                FOREIGN KEY(mid) REFERENCES Monkepo(mid));""")

dataframe = pd.read_csv("monkepo.csv")

count = 0
monkeType = {}
for row in dataframe.itertuples():
    if (row.name in monkeType):
        continue
    else:
        monkeType[row.name] = count
        cur.execute("INSERT INTO Monkepo (mid, name) VALUES (?, ?);", (count, row.name))
        cur.execute("INSERT INTO Class (mid, major, type) VALUES (?, 1, ?);", (count, row.majorclass))
        if row.minorclass != "None":
            cur.execute("INSERT INTO Class (mid, major, type) VALUES (?, 0, ?);", (count, row.minorclass))
        count+=1
        
for row in dataframe.itertuples():
        tempcount = monkeType[row.name]
        dt = datetime.datetime.strptime(row.date, "%Y-%m-%d").date()
        try:
            tm = datetime.datetime.strptime(row.time, "%H:%M:%S.%f").time()
        except ValueError as e:
            tm = datetime.datetime.strptime(row.time, "%H:%M:%S").time()
            continue
        comb = datetime.datetime.combine(dt, tm)
        
        try:
            cur.execute("INSERT INTO Appearance (mid, latitude, longitude, datetime) VALUES (?, ?, ?, ?);", (tempcount, row.latitude, row.longitude, comb))
        except sqlite3.IntegrityError as e:
            continue


monkepo.commit()
monkepo.close()

---
**Part 2:** Using the database that you created in part 1, write Python function to count the frequency of appearances of each Monkepo species. Your function should take the name of your database as an argument and return a sorted list of tuples (Monkepo.name, #appearances) with the most frequent first. Implement this function in using a dictionary like we did in Lecture 3.

In the cell below, enter the definition of your function, followed by a call to it, where the result is displayed via a Pandas data frame as shown in the code fragment supplied.

In [4]:
import sqlite3, operator, pandas
#Enter your Python 2 code here

def monkepoCount(str):
    monkepo = sqlite3.connect(str)
    cur = monkepo.cursor()

    listOfTuples={}
    #cur.execute("""SELECT name FROM Appearance JOIN Monkepo on Appearance.mid = Monkepo.mid""")

    dataframe = pandas.read_csv("monkepo.csv")
    
    for row in dataframe.itertuples():
        #print row[0]
        if (row[1] in listOfTuples):
            listOfTuples[row[1]] += 1
        else:
            listOfTuples[row[1]] = 1
            
    sorted_list = sorted(listOfTuples.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list

    
tupleList = monkepoCount("monkepo.db")

pandas.DataFrame(tupleList, columns=['Name', 'Count'])


,Name,Count
0,PigDye,24446
1,AttaRat,18662
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4767
6,TarPiece,4036
7,TubaZ,3641
8,ARasp,2593
9,AnIronAd-,2497


---
**Part 3:** Implement a second version of your function to count the frequency of appearances of each Monkepo species, but this time use SQL's "GROUP BY" directive. The input and output of this version should be the same as the one you implemented in Part 2. Display your answer using a Pandas data frame as well.

In [26]:
import sqlite3, operator

def monkepoCountGrp(str):
    monkepo = sqlite3.connect(str)
    cur = monkepo.cursor()

    listOfTuples={}
    cur.execute("""SELECT name, COUNT(name) 
                   FROM Appearance 
                   JOIN Monkepo on Appearance.mid = Monkepo.mid
                   GROUP BY name
                   ORDER BY COUNT(name)""")

    for row in cur:
        listOfTuples[row[0]] = row[1]

    sorted_list = sorted(listOfTuples.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list

    
tupleList = monkepoCountGrp("monkepo.db")
pandas.DataFrame(tupleList, columns=['Name', 'Count'])

,Name,Count
0,PigDye,24296
1,AttaRat,18557
2,RopeSaw,10320
3,DewEel,8651
4,Veeee,7279
5,NovaNet,4731
6,TarPiece,3995
7,TubaZ,3606
8,ARasp,2573
9,AnIronAd-,2460


---
**Part 4:** Use Jupyter's [*%timeit*](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) function to compare the performance of the two functions that you wrote in parts 2 and 3.

In [6]:
import pandas
import sqlite3
from timeit import default_timer as timer
#Enter your Python 2 code here

def monkepoCount(str):
    monkepo = sqlite3.connect(str)
    cur = monkepo.cursor()

    listOfTuples={}

    dataframe = pandas.read_csv("monkepo.csv")
    
    for row in dataframe.itertuples():
        if (row[1] in listOfTuples):
            listOfTuples[row[1]] += 1
        else:
            listOfTuples[row[1]] = 1
            
    sorted_list = sorted(listOfTuples.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list
    
def monkepoCountGrp(str):
    monkepo = sqlite3.connect(str)
    cur = monkepo.cursor()

    listOfTuples={}
    cur.execute("""SELECT name, COUNT(name) 
                   FROM Appearance 
                   JOIN Monkepo on Appearance.mid = Monkepo.mid
                   GROUP BY name
                   ORDER BY COUNT(name)""")

    for row in cur:
        listOfTuples[row[0]] = row[1]

    sorted_list = sorted(listOfTuples.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list

%timeit monkepoCount('monkepo.db')
%timeit monkepoCountGrp('monkepo.db')



1 loop, best of 3: 392 ms per loop
10 loops, best of 3: 192 ms per loop


---
**Part 5:** In the following cell write a Python function that finds both the rarest and most  common monkepo species for each class. Your function should take your database name as an input and it it should return a list with with a tuple for each distinct class type and columns with the most and least common Monkepo species names.

In [45]:
import pandas
import sqlite3



def monkepoCountGrp(str):
    monkepo = sqlite3.connect(str)
    cur = monkepo.cursor()
    
    common={}
    rare={}
    end={}
    count={}
    
    #cur.execute("""SELECT DISTINCT type FROM Class""")
    #for row in cur:
        #common[row[0]] = 'NaNCommon'
        #rare[row[0]] = 'NaNRare'
        #count[row[0]] = 0
        #print row
        

    
    
    cur.execute("""SELECT name, COUNT(name), C.type
                   FROM Appearance A
                   JOIN Monkepo M on A.mid = M.mid
                   JOIN Class C on C.mid = M.mid
                   WHERE C.major = 1
                   GROUP BY name
                   ORDER BY COUNT(name) DESC""")

    for row in cur:
        
        name = row[0]
        cnt = row[1]
        mpType = row[2]
        
        
        count[name] = cnt
        
        if mpType in common:
            
            if cnt > count[common[mpType]]:
                common[mpType] = name
                
        elif mpType not in common:
            common[mpType] = name
            
        if mpType in rare:            
            if cnt < count[rare[mpType]]:
                rare[mpType] = name
                
        elif mpType not in rare:
            rare[mpType] = name
            

    cur.execute("""SELECT name, COUNT(name), C.type
                   FROM Appearance A
                   JOIN Monkepo M on A.mid = M.mid
                   JOIN Class C on C.mid = M.mid
                   WHERE C.major = 0
                   GROUP BY name
                   ORDER BY COUNT(name) DESC""")

    for row in cur:
        #print row
        
        name = row[0]
        cnt = row[1]
        mpType = row[2]
        
        
        count[name] = cnt
        
        if mpType in common:
            
            if cnt > count[common[mpType]]:
                common[mpType] = name
                
        elif mpType not in common:
            common[mpType] = name
            
        if mpType in rare:            
            if cnt < count[rare[mpType]]:
                rare[mpType] = name
                
        elif mpType not in rare:
            rare[mpType] = name
        
    #for row in count:
        #print row, count[row]
    end['Most Common'] = common
    end['Most Rare'] = rare
    
    return end

    #print(df5)
    
type_list = monkepoCountGrp("monkepo.db")
pandas.DataFrame(type_list)

,Most Common,Most Rare
Aggressive,YankMe,IonHelment
Beastly,TrainId,RaidOrgan
Boring,PigDye,Asniper
Dirty,HerdSwans,XiNo
Elfish,ClayFire,WiffyGlutG
Freezing,Xjyn,Xjyn
Haunting,Stylag,Nagger
Hot,TigerHowl,AlienSent
Insect,DewEel,FerretTube
Metallic,GimmeATen,GimmeATen


---
**Part 6:** One of the reasons that Monkepo appear is that they depend on supply stations in order to survive in Earth's harsh environment. The same smartphone app that Millennials use for detecting Mokepos also detects the locations of these supply stations. A list of detected stations is available for <a href="http://csbio.unc.edu/mcmillan/Media/stations.csv" download>download</a> here.

In the next cell you should add a new table to your database called *SupplyStations*. Make sure that you commit and close the modified database.

Then write a code fragment that finds, for each supply station how many Monkepo appearances were closer to it than to any other station. Your code should produce a list of tuples (stationID, latitude, longitude, #ofNearbyMonkepo) sorted from most popular to least popular.

***This problem is challenging, allow for enough time to do it!*** You can approximate the distance between a Monkepo appearance and a supply station by $\sqrt{(latitude_{Monkepo} - latitude_{Station})^2 + (longitude_{Monkepo} - longitude_{Station})^2}$. This is not techically correct, but it is close at the scales and latitudes we are considering.

In [47]:
#Enter your Python 2 code 

import pandas, sqlite3, math, operator

monkepo = sqlite3.connect("monkepo.db")
cur = monkepo.cursor()

#cur.execute("""DROP TABLE SupplyStations""")
cur.execute("""CREATE TABLE IF NOT EXISTS SupplyStations (
                stationID INTEGER PRIMARY KEY, 
                latitude REAL,
                longitude REAL);""")
dataframe = pandas.read_csv("stations.csv")

for row in dataframe.itertuples():
    cur.execute("""INSERT OR IGNORE INTO SupplyStations (stationID, latitude, longitude) VALUES (?, ?, ?);""", (row[0], row[1], row[2]))



stations = {}
    
monkepo.commit()
monkepo.close()


#reconnects to find popular stations
monkepo = sqlite3.connect("monkepo.db")
cur = monkepo.cursor()

apps = []
cur.execute("""SELECT latitude, longitude FROM Appearance""")
for row in cur:
    apps.append(row)

stations = []
stationsDict={}
cur.execute("""SELECT stationID, latitude, longitude FROM SupplyStations""")
for row in cur:
    stations.append([row[0], row[1], row[2]])
    stationsDict[row[0]] = 0
#print pandas.DataFrame(stations)
    


#i = 0

for a in apps:
    #if i % 5000 == 0:
        #print i
    #i += 1
    dist = float('inf')
    clStation = -1
    for s in stations:
        d = math.sqrt((a[0]-s[1])**(2)+(a[1]-s[2])**(2))
        if d < dist:
            dist = d
            clStation = s[0]
        else:
            continue
    stationsDict[clStation] += 1


#print 'made it out of loop'
statTuples = []
#i = 0
for stat in stations:
    statTuples.append([stat[0], stat[1], stat[2], stationsDict[stat[0]]])
    #if i % 100 == 0:
        #print i, statTuples[-1]
    #i += 1
    
#print 'made it to sorting'
statTuples = sorted(statTuples, key=operator.itemgetter(3), reverse=True)

monkepo.close()
return statTuples

    

KeyboardInterrupt: 

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. We do not need your database, since your Notebook should produce it.